In [1]:
# ! conda install -c conda-forge spacy
# ! python -m spacy download en_core_web_sm

In [1]:
import spacy 
nlp = spacy.load('en_core_web_sm') 

In [11]:
# sentence = "What are total death cases in US on 27-09-2020?"
# sentence = "number of cities with total cases greater than 1000?"

def get_entities(sentence):
	doc = nlp(sentence) 
	print("----> Entities:")
	for ent in doc.ents: 
		print("-------->",ent.text, ent.start_char, ent.end_char, ent.label_) 

def get_tokens(sentence):
	doc = nlp(sentence) 
	print("----> Tokens:")
	for token in doc: 
		print("-------->",token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop) 

In [20]:
queries=[]
with open("../possible-questions.txt","r") as f:
    for line in f.readlines():
        line = line[:-1]
        if(line):
            queries.append(line)

In [21]:
for sent in queries:
    print(sent)
    get_entities(sent)
    get_tokens(sent)


total number of cases found in Afganistan?
----> Entities:
--------> Afganistan 31 41 GPE
----> Tokens:
--------> total total ADJ JJ amod xxxx True False
--------> number number NOUN NN ROOT xxxx True False
--------> of of ADP IN prep xx True True
--------> cases case NOUN NNS pobj xxxx True False
--------> found find VERB VBN acl xxxx True False
--------> in in ADP IN prep xx True True
--------> Afganistan Afganistan PROPN NNP pobj Xxxxx True False
--------> ? ? PUNCT . punct ? False False
total number of cases found in Colombia till july?
----> Entities:
--------> Colombia 31 39 GPE
--------> july 45 49 DATE
----> Tokens:
--------> total total ADJ JJ amod xxxx True False
--------> number number NOUN NN ROOT xxxx True False
--------> of of ADP IN prep xx True True
--------> cases case NOUN NNS pobj xxxx True False
--------> found find VERB VBN acl xxxx True False
--------> in in ADP IN prep xx True True
--------> Colombia Colombia PROPN NNP pobj Xxxxx True False
--------> till till SC